# Basic Optimization
In this example, we'll be performing a simple optimization of single-objective functions using the global-best optimizer in `pyswarms.single.GBestPSO` and the local-best optimizer in `pyswarms.single.LBestPSO`. This aims to demonstrate the basic capabilities of the library when applied to benchmark problems.

In [1]:
import sys
# Change directory to access the pyswarms module
sys.path.append('../')

In [2]:
print('Running on Python version: {}'.format(sys.version))

Running on Python version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [3]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Optimizing a function
First, let's start by optimizing the sphere function. Recall that the minima of this function can be located at `f(0,0..,0)` with a value of `0`. In case you don't remember the characteristics of a given function, simply call `help(<function>)`.

For now let's just set some arbitrary parameters in our optimizers. There are, at minimum, three steps to perform optimization:

1. Set the hyperparameters to configure the swarm as a `dict`.
2. Create an instance of the optimizer by passing the dictionary along with the necessary arguments.
3. Call the `optimize()` method and have it store the optimal cost and position in a variable.

The `optimize()` method returns a `tuple` of values, one of which includes the optimal cost and position after optimization. You can store it in a single variable and just index the values, or unpack it using several variables at once.

In [20]:
%%time
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)

# Perform optimization
cost, pos = optimizer.optimize(fx.sphere, iters=1000)

2019-02-21 00:13:40,666 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}





pyswarms.single.global_best:   0%|          |0/1000




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.00361




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.00361




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.00361




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.00361




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.00361




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.000613




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.000613




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.000613




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.000613




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.000613




pyswarms.single.global_best:   1%|          |7/1000, best_c

pyswarms.single.global_best:   7%|▋         |73/1000, best_cost=3.48e-6




pyswarms.single.global_best:   7%|▋         |73/1000, best_cost=3.48e-6




pyswarms.single.global_best:   7%|▋         |73/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=3.48e-6




pyswarms.single.global_best:   9%|▊         |87/1000, best_cost=3.48e-6




pyswarms.single.global_best:   9%|▊         |87/1000, best_cost=3.48e-6




pyswarms.sin

pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=7.3e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=7.3e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=7.3e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=7.3e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=7.3e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=7.3e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=2.33e-10




pyswarms.single.global_best:  16%|█▌        |155/1000, best_cost=2.33e-10




pyswarms.single.global_best:  16%|█▋        |163/1000, best_cost=2.33e-10




pyswarms.single.global_best:  16%|█▋        |163/1000, best_cost=2.33e-10




pyswarms.single.global_best:  16%|█▋        |163/1000, best_cost=2.33e-10




pyswarms.single.global_best:  16%|█▋        |163/1000, best_cost=2.33e-10




pyswarms.single.global_best:  16%|█▋        |163/1000, best_cost=2.33e

pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=7.23e-14




pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=7.23e-14




pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=7.23e-14




pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=7.23e-14




pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=1.35e-14




pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=1.35e-14




pyswarms.single.global_best:  23%|██▎       |232/1000, best_cost=1.35e-14




pyswarms.single.global_best:  24%|██▍       |240/1000, best_cost=1.35e-14




pyswarms.single.global_best:  24%|██▍       |240/1000, best_cost=1.35e-14




pyswarms.single.global_best:  24%|██▍       |240/1000, best_cost=1.35e-14




pyswarms.single.global_best:  24%|██▍       |240/1000, best_cost=1.35e-14




pyswarms.single.global_best:  24%|██▍       |240/1000, best_cost=1.35e-14




pyswarms.single.global_best:  24%|██▍       |240/1000, best_cost

pyswarms.single.global_best:  31%|███       |309/1000, best_cost=3.42e-19




pyswarms.single.global_best:  31%|███       |309/1000, best_cost=3.42e-19




pyswarms.single.global_best:  31%|███       |309/1000, best_cost=3.42e-19




pyswarms.single.global_best:  31%|███       |309/1000, best_cost=3.42e-19




pyswarms.single.global_best:  31%|███       |309/1000, best_cost=3.42e-19




pyswarms.single.global_best:  31%|███       |309/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost=3.42e-19




pyswarms.single.global_best:  32%|███▏      |317/1000, best_cost

...

pyswarms.single.global_best:  70%|██████▉   |695/1000, best_cost=6.55e-32




pyswarms.single.global_best:  70%|██████▉   |695/1000, best_cost=4.93e-32




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=4.93e-32




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=2.26e-32




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=3.36e-33




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=3.36e-33




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=3.36e-33




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=3.36e-33




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=3.36e-33




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=2.75e-33




pyswarms.single.global_best:  70%|███████   |703/1000, best_cost=2.75e-33




pyswarms.single.global_best:  71%|███████   |711/1000, best_cost=2.75e-33




pyswarms.single.global_best:  71%|███████   |711/1000, best_cost

pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=2.03e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=2.03e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=2.03e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=2.03e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=2.03e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=2.03e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=1.43e-36




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=1.43e-36




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=1.43e-36




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=1.43e-36




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=1.43e-36




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=1.43e-36




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost

pyswarms.single.global_best:  85%|████████▌ |850/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  86%|████████▌ |858/1000, best_cost=9.73e-41




pyswarms.single.global_best:  87%|████████▋ |866/1000, best_cost=9.73e-41




pyswarms.single.global_best:  87%|████████▋ |866/1000, best_cost=8.71e-41




pyswarms.single.global_best:  87%|████████▋ |866/1000, best_cost

pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=1.42e-42




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=1.42e-42




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=1.42e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.42e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.41e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.41e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.41e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.4e-42 




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.4e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.4e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.4e-42




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=1.4e-42




pyswarms.single.global_best:  95%|█████████▍|946/1000, best_cost=1.4

CPU times: user 5.29 s, sys: 681 ms, total: 5.97 s
Wall time: 14.5 s


We can see that the optimizer was able to find a good minima as shown above. You can control the verbosity of the output using the `verbose` argument, and the number of steps to be printed out using the `print_step` argument.

Now, let's try this one using local-best PSO:

In [8]:
%%time
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9, 'k': 2, 'p': 2}

# Call instance of PSO
optimizer = ps.single.LocalBestPSO(n_particles=10, dimensions=2, options=options)

# Perform optimization
cost, pos = optimizer.optimize(fx.sphere, iters=1000)

2019-02-21 00:10:16,007 - pyswarms.single.local_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}


pyswarms.single.local_best:   0%|          |0/1000

pyswarms.single.local_best:   0%|          |0/1000, best_cost=0.139

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.139

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.139

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.139

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.139

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.139

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.138

pyswarms.single.local_best:   0%|          |1/1000, best_cost=0.138

pyswarms.single.local_best:   1%|          |7/1000, best_cost=0.138

pyswarms.single.local_best:   1%|          |7/1000, best_cost=0.084

pyswarms.single.local_best:   1%|          |7/1000, best_cost=0.0425

pyswarms.single.local_best:   1%|          |7/

pyswarms.single.local_best:   9%|▊         |86/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▊         |86/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▊         |86/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:   9%|▉         |93/1000, best_cost=1.5e-7

pyswarms.single.local_best:  10%|█         |100/1000, best_cost=1.5e-7

pyswarms.single.local_best:  10%|█         |100/1000, best_cost=1.5e-7

pyswarms.single.local_best:  10%|█         |100/1000, best_cost=1.5e-7

pys

pyswarms.single.local_best:  17%|█▋        |174/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  18%|█▊        |182/1000, best_cost=1.75e-11

pyswarms.single.local_best:  19%|█▉        |189/1000, best_cost=1.75e-11

pyswarms.single.local_best:  19%|█▉        |189/1000, best_cost=1.75e-11

pyswarms.single.local_best:  19%|█▉        |189/1000, best_cost=1.75e-11

pyswarms.single.local_best:  19%|█▉        |189/1000, best_cost=1.72e-11

pyswarms.single.local_best:  19%|█▉   

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  27%|██▋       |270/1000, best_cost=1.72e-16

pyswarms.single.local_best:  28%|██▊       |278/1000, best_cost=1.72e-16

pyswarms.single.local_best:  28%|██▊       |278/1000, best_cost=1.72e-16

pyswarms.single.local_best:  28%|██▊       |278/1000, best_cost=1.72e-16

pyswarms.single.local_best:  28%|██▊       |278/1000, best_cost=1.72e-16

pyswarms.single.local_best:  28%|██▊       |278/1000, best_cost=1.72e-16

pyswarms.single.local_best:  28%|██▊  

pyswarms.single.local_best:  36%|███▌      |358/1000, best_cost=4.71e-18

pyswarms.single.local_best:  36%|███▌      |358/1000, best_cost=8.81e-19

pyswarms.single.local_best:  36%|███▌      |358/1000, best_cost=8.81e-19

pyswarms.single.local_best:  36%|███▌      |358/1000, best_cost=8.81e-19

pyswarms.single.local_best:  36%|███▌      |358/1000, best_cost=8.81e-19

pyswarms.single.local_best:  36%|███▌      |358/1000, best_cost=8.81e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=8.81e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=8.81e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=8.81e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=8.81e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=8.81e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=4.85e-19

pyswarms.single.local_best:  37%|███▋      |366/1000, best_cost=4.85e-19

pyswarms.single.local_best:  37%|███▋ 

...

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=2.38e-32

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=2.38e-32

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=1.16e-32

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=1.16e-32

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=1.16e-32

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=9.83e-33

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=5.64e-33

pyswarms.single.local_best:  72%|███████▏  |721/1000, best_cost=5.04e-33

pyswarms.single.local_best:  73%|███████▎  |729/1000, best_cost=5.04e-33

pyswarms.single.local_best:  73%|███████▎  |729/1000, best_cost=5.04e-33

pyswarms.single.local_best:  73%|███████▎  |729/1000, best_cost=5.04e-33

pyswarms.single.local_best:  73%|███████▎  |729/1000, best_cost=5.04e-33

pyswarms.single.local_best:  73%|███████▎  |729/1000, best_cost=5.04e-33

pyswarms.single.local_best:  73%|█████

pyswarms.single.local_best:  81%|████████  |808/1000, best_cost=4.28e-37

pyswarms.single.local_best:  81%|████████  |808/1000, best_cost=4.28e-37

pyswarms.single.local_best:  81%|████████  |808/1000, best_cost=1.63e-37

pyswarms.single.local_best:  81%|████████  |808/1000, best_cost=1.51e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.51e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.51e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.51e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.51e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.51e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.08e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.08e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.08e-37

pyswarms.single.local_best:  82%|████████▏ |816/1000, best_cost=1.08e-37

pyswarms.single.local_best:  82%|█████

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  90%|█████████ |902/1000, best_cost=7.67e-41

pyswarms.single.local_best:  91%|█████████ |910/1000, best_cost=7.67e-41

pyswarms.single.local_best:  91%|█████████ |910/1000, best_cost=7.67e-41

pyswarms.single.local_best:  91%|█████████ |910/1000, best_cost=7.67e-41

pyswarms.single.local_best:  91%|█████████ |910/1000, best_cost=7.67e-41

pyswarms.single.local_best:  91%|█████████ |910/1000, best_cost=7.67e-41

pyswarms.single.local_best:  91%|█████

pyswarms.single.local_best:  99%|█████████▊|987/1000, best_cost=1.17e-42

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=1.17e-42

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=9.82e-43

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=7.06e-43

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=2.84e-43

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=1.24e-43

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=1.24e-43

pyswarms.single.local_best:  99%|█████████▉|994/1000, best_cost=1.24e-43

pyswarms.single.local_best: 100%|██████████|1000/1000, best_cost=1.24e-432019-02-21 00:10:32,214 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 1.238324896558579e-43, best pos: [3.29854417e-22 4.16052549e-22]


CPU times: user 5.77 s, sys: 689 ms, total: 6.46 s
Wall time: 16.2 s


## Optimizing a function with bounds
Another thing that we can do is to set some bounds into our solution, so as to contain our candidate solutions within a specific range. We can do this simply by passing a `bounds` parameter, of type `tuple`, when creating an instance of our swarm. Let's try this using the global-best PSO with the Rastrigin function (`rastrigin` in `pyswarms.utils.functions.single_obj`).

Recall that the Rastrigin function is bounded within `[-5.12, 5.12]`. If we pass an unbounded swarm into this function, then a `ValueError` might be raised. So what we'll do is to create a bound within the specified range. There are some things to remember when specifying a bound:

* A bound should be of type tuple with length 2. 
* It should contain two `numpy.ndarrays` so that we have a `(min_bound, max_bound)`
* Obviously, all values in the `max_bound` should always be greater than the `min_bound`. Their shapes should match the dimensions of the swarm.

What we'll do now is to create a 10-particle, 2-dimensional swarm. This means that we have to set our maximum and minimum boundaries with the shape of 2. In case we want to initialize an n-dimensional swarm, we then have to set our bounds with the same shape n. A fast workaround for this would be to use the `numpy.ones` function multiplied by a constant.

In [9]:
# Create bounds
max_bound = 5.12 * np.ones(2)
min_bound = - max_bound
bounds = (min_bound, max_bound)

In [12]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO with bounds argument
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options, bounds=bounds)

# Perform optimization
cost, pos = optimizer.optimize(fx.rastrigin,  iters=1000)

2019-02-21 00:10:50,014 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}




pyswarms.single.global_best:   0%|          |0/1000



pyswarms.single.global_best:   0%|          |0/1000, best_cost=20.1



pyswarms.single.global_best:   0%|          |0/1000, best_cost=18.4



pyswarms.single.global_best:   0%|          |0/1000, best_cost=9.73



pyswarms.single.global_best:   0%|          |0/1000, best_cost=9.73



pyswarms.single.global_best:   0%|          |0/1000, best_cost=8.11



pyswarms.single.global_best:   0%|          |0/1000, best_cost=8.11



pyswarms.single.global_best:   0%|          |0/1000, best_cost=8.11



pyswarms.single.global_best:   0%|          |0/1000, best_cost=8.11



pyswarms.single.global_best:   1%|          |8/1000, best_cost=8.11



pyswarms.single.global_best:   1%|          |8/1000, best_cost=6.66



pyswarms.single.global_best:   1%|          |8/1000, best_cost=6.66



pyswarms.single.global_best:   1%| 

pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.00459



pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.00459



pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.00153



pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.00153



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.00153



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.00153



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.00153



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.000378



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.000378



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.000378



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.000378



pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.000378



pyswarms.single.global_best:   9%|▉         |93/1000, best_cost=0.000378



pyswarms.single.glo

pyswarms.single.global_best:  16%|█▌        |157/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=2.16e-7



pyswarms.single.global_best:  17%|█▋        |172/1000, best_cost=2.16e-7



pyswarms.single.global_best:  17%|█▋        |172/1000, best_cost=2.16e-7



pyswarms.single.global_best:  17%|█▋        |172/1000, best_cost=2.16e-7



pyswarms.single.global_best:  17%|█▋        |172/1000, best_cost=2.16e-7



pyswarms.sin

pyswarms.single.global_best:  24%|██▍       |238/1000, best_cost=3.63e-10



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=3.63e-10



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=3.63e-10



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=3.63e-10



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=3.63e-10



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=2.07e-10



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=5.8e-11 



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=5.8e-11



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=5.8e-11



pyswarms.single.global_best:  25%|██▍       |246/1000, best_cost=5.8e-11



pyswarms.single.global_best:  25%|██▌       |254/1000, best_cost=5.8e-11



pyswarms.single.global_best:  25%|██▌       |254/1000, best_cost=5.8e-11



pyswarms.single.global_best:  25%|██▌       |254/1000, best_cost=5.8e-11



pyswa

pyswarms.single.global_best:  32%|███▏      |321/1000, best_cost=2.91e-13



pyswarms.single.global_best:  32%|███▏      |321/1000, best_cost=2.91e-13



pyswarms.single.global_best:  32%|███▏      |321/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  33%|███▎      |329/1000, best_cost=2.84e-13



pyswarms.single.global_best:  34%|███▎      |337/1000, best_cost=2.84e-13




...

pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  66%|██████▌   |662/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.global_best:  67%|██████▋   |670/1000, best_cost=0



pyswarms.single.glob

pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  75%|███████▍  |749/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.global_best:  76%|███████▌  |757/1000, best_cost=0



pyswarms.single.glob

pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |838/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |845/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |845/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |845/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |845/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |845/1000, best_cost=0



pyswarms.single.global_best:  84%|████████▍ |845/1000, best_cost=0



pyswarms.single.glob

pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  92%|█████████▎|925/1000, best_cost=0



pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=0



pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=0



pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=0



pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=0



pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=0



pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=0



pyswarms.single.glob

CPU times: user 5.59 s, sys: 800 ms, total: 6.39 s
Wall time: 15 s


## Basic Optimization with Arguments

Here, we will run a basic optimization using an objective function that needs parameterization.  We will use the ``single.GBestPSO`` and a version of the rosenbrock function to demonstrate

In [13]:
import sys
# change directory to access pyswarms
sys.path.append('../')

print("Running Python {}".format(sys.version))

Running Python 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [14]:
# import modules
import numpy as np

# create a parameterized version of the classic Rosenbrock unconstrained optimzation function
def rosenbrock_with_args(x, a, b, c=0):

    f = (a - x[:, 0]) ** 2 + b * (x[:, 1] - x[:, 0] ** 2) ** 2 + c
    return f

### Using Arguments

Arguments can either be passed in using a tuple or a dictionary, using the ``kwargs={}`` paradigm. First lets optimize the Rosenbrock function using keyword arguments.  Note in the definition of the Rosenbrock function above, there were two arguments that need to be passed other than the design variables, and one optional keyword argument, ``a``, ``b``, and ``c``, respectively

In [16]:
from pyswarms.single.global_best import GlobalBestPSO

# instatiate the optimizer
x_max = 10 * np.ones(2)
x_min = -1 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = GlobalBestPSO(n_particles=10, dimensions=2, options=options, bounds=bounds)

# now run the optimization, pass a=1 and b=100 as a tuple assigned to args

cost, pos = optimizer.optimize(rosenbrock_with_args, 1000, a=1, b=100, c=0)

2019-02-21 00:11:25,457 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}





pyswarms.single.global_best:   0%|          |0/1000




pyswarms.single.global_best:   0%|          |0/1000, best_cost=331




pyswarms.single.global_best:   0%|          |0/1000, best_cost=331




pyswarms.single.global_best:   0%|          |0/1000, best_cost=14.4




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.064




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.064




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.064




pyswarms.single.global_best:   0%|          |0/1000, best_cost=0.064




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.064




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.064




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.064




pyswarms.single.global_best:   1%|          |7/1000, best_cost=0.064




pyswarms.single.

pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=0.0098




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.0098




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.0098




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.0098




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.00971




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=0.00971




pyswarms.single.global_

pyswarms.single.global_best:  15%|█▌        |151/1000, best_cost=0.000222




pyswarms.single.global_best:  15%|█▌        |151/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost=0.000222




pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost

pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=0.000138




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=0.000138




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=0.000138




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=0.000135




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=0.000135




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost

pyswarms.single.global_best:  31%|███       |308/1000, best_cost=6.56e-5




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=6.22e-5




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.97e-5




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.8e-5 




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.7e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.7e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.63e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.5e-5 




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.34e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.24e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.17e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.12e-5




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=4.98e-5




p

...

pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=3.09e-14




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=3.09e-14




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  63%|██████▎   |627/1000, best_cost=3.09e-14




pyswarms.single.global_best:  64%|██████▎   |635/1000, best_cost

pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=8.27e-16




pyswarms.single.global_best:  71%|███████   |710/1000, best_cost=8.27e-16




pyswarms.single.global_best:  71%|███████   |710/1000, best_cost=8.27e-16




pyswarms.single.global_best:  71%|███████   |710/1000, best_cost=8.27e-16




pyswarms.single.global_best:  71%|███████   |710/1000, best_cost

pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=5.16e-18




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=5.16e-18




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=3.87e-18




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=3.87e-18




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=3.87e-18




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=3.87e-18




pyswarms.single.global_best:  78%|███████▊  |778/1000, best_cost=3.87e-18




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=3.87e-18




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=3.87e-18




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=3.87e-18




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=3.87e-18




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost=2.5e-18 




pyswarms.single.global_best:  79%|███████▊  |786/1000, best_cost

pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=7.14e-19




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=7.14e-19




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=7.03e-19




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=6.93e-19




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=6.93e-19




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost

pyswarms.single.global_best:  93%|█████████▎|929/1000, best_cost=8.18e-20




pyswarms.single.global_best:  93%|█████████▎|929/1000, best_cost=7.72e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=7.72e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=6.82e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=5.3e-20 




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=4.12e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=3.2e-20 




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=2.49e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=1.95e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=1.54e-20




pyswarms.single.global_best:  94%|█████████▎|937/1000, best_cost=1.15e-20




pyswarms.single.global_best:  94%|█████████▍|945/1000, best_cost=1.15e-20




pyswarms.single.global_best:  94%|█████████▍|945/1000, best_cost

It is also possible to pass a dictionary of key word arguments by using ``**`` decorator when passing the dict

In [17]:
kwargs={"a": 1.0, "b": 100.0, 'c':0}
cost, pos = optimizer.optimize(rosenbrock_with_args, 1000, **kwargs)

2019-02-21 00:11:40,201 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}





pyswarms.single.global_best:   0%|          |0/1000




pyswarms.single.global_best:   0%|          |0/1000, best_cost=1.69e-23




pyswarms.single.global_best:   0%|          |0/1000, best_cost=1.69e-23




pyswarms.single.global_best:   0%|          |0/1000, best_cost=1.69e-23




pyswarms.single.global_best:   0%|          |0/1000, best_cost=1.69e-23




pyswarms.single.global_best:   0%|          |0/1000, best_cost=1.69e-23




pyswarms.single.global_best:   0%|          |0/1000, best_cost=1.37e-23




pyswarms.single.global_best:   0%|          |0/1000, best_cost=4.67e-25




pyswarms.single.global_best:   0%|          |0/1000, best_cost=4.67e-25




pyswarms.single.global_best:   1%|          |8/1000, best_cost=4.67e-25




pyswarms.single.global_best:   1%|          |8/1000, best_cost=4.67e-25




pyswarms.single.global_best:   1%|          |8/1000, b

pyswarms.single.global_best:   7%|▋         |71/1000, best_cost=1.59e-26




pyswarms.single.global_best:   7%|▋         |71/1000, best_cost=1.59e-26




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=1.59e-26




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=1.59e-26




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=1.59e-26




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=4.31e-27




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=4.31e-27




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=4.31e-27




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=4.31e-27




pyswarms.single.global_best:   8%|▊         |79/1000, best_cost=4.31e-27




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=4.31e-27




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=4.31e-27




pyswarms.single.global_best:   9%|▊         |86/1000, best_cost=1.91e-27





pyswarms.single.global_best:  15%|█▌        |151/1000, best_cost=2.02e-28




pyswarms.single.global_best:  15%|█▌        |151/1000, best_cost=2.02e-28




pyswarms.single.global_best:  15%|█▌        |151/1000, best_cost=2.02e-28




pyswarms.single.global_best:  15%|█▌        |151/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=2.02e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=1.94e-28




pyswarms.single.global_best:  16%|█▌        |158/1000, best_cost=1.94e-28




pyswarms.single.global_best:  16%|█▋        |165/1000, best_cost

pyswarms.single.global_best:  22%|██▎       |225/1000, best_cost=1.6e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.6e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.6e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.6e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.59e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.59e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.59e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.59e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.57e-28




pyswarms.single.global_best:  23%|██▎       |233/1000, best_cost=1.57e-28




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=1.57e-28




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=1.57e-28




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=1.5

pyswarms.single.global_best:  31%|███       |306/1000, best_cost=1.4e-28 




pyswarms.single.global_best:  31%|███       |306/1000, best_cost=1.4e-28




pyswarms.single.global_best:  31%|███       |306/1000, best_cost=1.37e-28




pyswarms.single.global_best:  31%|███       |306/1000, best_cost=1.36e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.36e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.36e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.35e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.31e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.29e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.28e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.26e-28




pyswarms.single.global_best:  31%|███▏      |313/1000, best_cost=1.23e-28




pyswarms.single.global_best:  32%|███▏      |320/1000, best_cost=

...

pyswarms.single.global_best:  62%|██████▏   |617/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |617/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |617/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |617/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |617/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |617/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▎   |625/1000, best_cost

pyswarms.single.global_best:  69%|██████▉   |694/1000, best_cost=5.87e-29




pyswarms.single.global_best:  69%|██████▉   |694/1000, best_cost=5.87e-29




pyswarms.single.global_best:  69%|██████▉   |694/1000, best_cost=5.87e-29




pyswarms.single.global_best:  69%|██████▉   |694/1000, best_cost=5.87e-29




pyswarms.single.global_best:  69%|██████▉   |694/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |702/1000, best_cost

pyswarms.single.global_best:  77%|███████▋  |773/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |773/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |773/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |773/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |773/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |773/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost

pyswarms.single.global_best:  85%|████████▍ |846/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▍ |846/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▌ |854/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |862/1000, best_cost

pyswarms.single.global_best:  92%|█████████▏|924/1000, best_cost=5.87e-29




pyswarms.single.global_best:  92%|█████████▏|924/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|932/1000, best_cost=5.87e-29




pyswarms.single.global_best:  94%|█████████▍|940/1000, best_cost=5.87e-29




pyswarms.single.global_best:  94%|█████████▍|940/1000, best_cost

Any key word arguments in the objective function can be left out as they will be passed the default as defined in the prototype. Note here, ``c`` is not passed into the function.

In [18]:
cost, pos = optimizer.optimize(rosenbrock_with_args, 1000, a=1, b=100)

2019-02-21 00:11:55,073 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}





pyswarms.single.global_best:   0%|          |0/1000




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   0%|          |0/1000, best_cost=5.87e-29




pyswarms.single.global_best:   1%|          |8/1000, best_cost=5.87e-29




pyswarms.single.global_best:   1%|          |8/1000, best_cost=5.87e-29




pyswarms.single.global_best:   1%|          |8/1000, b

pyswarms.single.global_best:   7%|▋         |73/1000, best_cost=5.87e-29




pyswarms.single.global_best:   7%|▋         |73/1000, best_cost=5.87e-29




pyswarms.single.global_best:   7%|▋         |73/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   8%|▊         |80/1000, best_cost=5.87e-29




pyswarms.single.global_best:   9%|▊         |87/1000, best_cost=5.87e-29




pyswarms.single.global_best:   9%|▊         |87/1000, best_cost=5.87e-29





pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  15%|█▌        |153/1000, best_cost=5.87e-29




pyswarms.single.global_best:  16%|█▌        |161/1000, best_cost=5.87e-29




pyswarms.single.global_best:  16%|█▌        |161/1000, best_cost=5.87e-29




pyswarms.single.global_best:  16%|█▌        |161/1000, best_cost=5.87e-29




pyswarms.single.global_best:  16%|█▌        |161/1000, best_cost=5.87e-29




pyswarms.single.global_best:  16%|█▌        |161/1000, best_cost=5.87e-29




pyswarms.single.global_best:  16%|█▌        |161/1000, best_cost

pyswarms.single.global_best:  23%|██▎       |227/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |227/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  23%|██▎       |234/1000, best_cost=5.87e-29




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=5.87e-29




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost=5.87e-29




pyswarms.single.global_best:  24%|██▍       |241/1000, best_cost

pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  31%|███       |308/1000, best_cost=5.87e-29




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.87e-29




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.87e-29




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.87e-29




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.87e-29




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost=5.87e-29




pyswarms.single.global_best:  32%|███▏      |316/1000, best_cost

...

pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  62%|██████▏   |619/1000, best_cost=5.87e-29




pyswarms.single.global_best:  63%|██████▎   |626/1000, best_cost=5.87e-29




pyswarms.single.global_best:  63%|██████▎   |626/1000, best_cost=5.87e-29




pyswarms.single.global_best:  63%|██████▎   |626/1000, best_cost=5.87e-29




pyswarms.single.global_best:  63%|██████▎   |626/1000, best_cost=5.87e-29




pyswarms.single.global_best:  63%|██████▎   |626/1000, best_cost=5.87e-29




pyswarms.single.global_best:  63%|██████▎   |626/1000, best_cost

pyswarms.single.global_best:  69%|██████▉   |690/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|██████▉   |698/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |705/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |705/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |705/1000, best_cost=5.87e-29




pyswarms.single.global_best:  70%|███████   |705/1000, best_cost

pyswarms.single.global_best:  77%|███████▋  |772/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |772/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |772/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |772/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |772/1000, best_cost=5.87e-29




pyswarms.single.global_best:  77%|███████▋  |772/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost=5.87e-29




pyswarms.single.global_best:  78%|███████▊  |780/1000, best_cost

pyswarms.single.global_best:  85%|████████▍ |848/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▍ |848/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▍ |848/1000, best_cost=5.87e-29




pyswarms.single.global_best:  85%|████████▍ |848/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost=5.87e-29




pyswarms.single.global_best:  86%|████████▌ |856/1000, best_cost

pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  93%|█████████▎|930/1000, best_cost=5.87e-29




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=5.87e-29




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=5.87e-29




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost=5.87e-29




pyswarms.single.global_best:  94%|█████████▍|938/1000, best_cost